# Data Augmentation For DataSet2 

Because using `Fast.AI`'s Pytorch Code so we need to have a global tracking ID for data augmentation

In [3]:
import os
os.getcwd()

'/Users/mingrenshen/Projects/MedicalImgAnalysis/notebooks'

In [4]:
import imgaug as ia
from imgaug import augmenters as iaa
import skimage.io
import errno
import numpy as np
import skimage.color as color
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.color import gray2rgb

In [5]:
def getImageList(imageTXT):
    """
    Function to loop the testing images for test
    :param imageTXT: the txt that stores the 
    :return: imageFileList: the list contains all the original test image list
    """
    imageFileList = list()
    with open(imageTXT,'r') as f:
        lines = f.readlines()
        for line in lines:
            imageFileList.append(line.strip())
    return imageFileList

In [6]:
def createFolder(folderName):
    """
    Safely create folder when needed
    
    :param folderName : the directory that you  want to safely create
    :return: None
    """
    if not os.path.exists(folderName):
        try:
            os.makedirs(folderName)
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [7]:
# Starting ID for Bleeding and non Bleeding
# Get from the Log File when Generating Training Images
startIDBleeding = 773
startIDnonBleeding = 391
# This cell should only be run once

In [8]:
dataPath = "../data/DataSet9/fastAITrain/train/"

In [9]:
# (1) Gaussion Noise - 1

##################################################
# 1. Define data augmentation operations for bleeding
##################################################

## Processing Bleeding
trainImageTxtFile = dataPath + "bleeding.txt"
current_class = 'bleeding'

imageList = getImageList(trainImageTxtFile)

current_operation = "GaussianNoise"

# Add gaussian noise.
# For 50% of all images, we sample the noise once per pixel.
# For the other 50% of all images, we sample the noise per pixel AND
# channel. This can change the color (not only brightness) of the
# pixels.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(loc=0, 
                                  scale=(0.0, 0.05*255), 
                                  per_channel=0.5)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    # print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDBleeding) + '.jpg',image_aug)
    startIDBleeding += 1

In [10]:
# (1) Gaussion Noise - 2

##################################################
# 1. Define data augmentation operations for nonbleeding
##################################################

## Processing nonBleeding

trainImageTxtFile = dataPath + "nonbleeding.txt"
current_class = 'nonbleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "GaussianNoise"

# Add gaussian noise.
# For 50% of all images, we sample the noise once per pixel.
# For the other 50% of all images, we sample the noise per pixel AND
# channel. This can change the color (not only brightness) of the
# pixels.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(loc=0, 
                                  scale=(0.0, 0.05*255), 
                                  per_channel=0.5)
])


# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    # print(img)
    # Grayscale images must have shape (height, width, 1) each.
    #print(os.listdir(dataPath+'images/'))
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    # bbs = bboxSetupInImage(dataPath , img.rstrip('.jpg') + '.txt',currentimage)
    # Augment BBs and images.
    # As we only have one image and list of BBs, we use
    # [image] and [bbs] to turn both into lists (batches) for the# functions and then [0] to reverse that. In a real experiment, your
    # variables would likely already be lists.
    image_aug = seq_det.augment_images([currentimage])[0]
    # bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    augImgFolder = current_operation + "Images_" + current_class
    # augTxTFolder = current_operation + "TXT"
    createFolder(augImgFolder)
    # createFolder(augTxTFolder)
    # Save aug images and bboxes
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDnonBleeding) + '.jpg',image_aug)
    startIDnonBleeding += 1
print('Done')

Done


In [11]:
# (2) Gaussion Blur - 1 

##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "bleeding.txt"
current_class = 'bleeding'

imageList = getImageList(trainImageTxtFile)

current_operation = "GaussianBlur"

# blur images with a sigma of 0 to 3.0  
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
  iaa.GaussianBlur(sigma=(0, 3)) 
])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDBleeding) + '.jpg',image_aug)
    startIDBleeding += 1
print('Done')

Done


In [12]:
# (2) Gaussion Blur - 2

##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "nonbleeding.txt"
current_class = 'nonbleeding'

imageList = getImageList(trainImageTxtFile)

current_operation = "GaussianBlur"

# blur images with a sigma of 0 to 3.0  
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
  iaa.GaussianBlur(sigma=(0, 3)) 
])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDnonBleeding) + '.jpg',image_aug)
    startIDnonBleeding += 1
print('Done')

nonbleeding.1.jpg
nonbleeding.10.jpg
nonbleeding.100.jpg
nonbleeding.101.jpg
nonbleeding.102.jpg
nonbleeding.103.jpg
nonbleeding.104.jpg
nonbleeding.105.jpg
nonbleeding.106.jpg
nonbleeding.107.jpg
nonbleeding.108.jpg
nonbleeding.109.jpg
nonbleeding.11.jpg
nonbleeding.110.jpg
nonbleeding.111.jpg
nonbleeding.112.jpg
nonbleeding.113.jpg
nonbleeding.114.jpg
nonbleeding.115.jpg
nonbleeding.116.jpg
nonbleeding.117.jpg
nonbleeding.118.jpg
nonbleeding.119.jpg
nonbleeding.12.jpg
nonbleeding.120.jpg
nonbleeding.121.jpg
nonbleeding.122.jpg
nonbleeding.123.jpg
nonbleeding.124.jpg
nonbleeding.125.jpg
nonbleeding.126.jpg
nonbleeding.127.jpg
nonbleeding.128.jpg
nonbleeding.129.jpg
nonbleeding.13.jpg
nonbleeding.130.jpg
nonbleeding.131.jpg
nonbleeding.132.jpg
nonbleeding.133.jpg
nonbleeding.134.jpg
nonbleeding.135.jpg
nonbleeding.136.jpg
nonbleeding.137.jpg
nonbleeding.138.jpg
nonbleeding.139.jpg
nonbleeding.14.jpg
nonbleeding.140.jpg
nonbleeding.141.jpg
nonbleeding.142.jpg
nonbleeding.143.jpg
nonblee

In [13]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "bleeding.txt"
current_class = 'bleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "Brightness"

# Strengthen or weaken the contrast in each image.   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5))
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDBleeding) + '.jpg',image_aug)
    startIDBleeding += 1
print('Done')

bleeding.1.jpg
bleeding.10.jpg
bleeding.100.jpg
bleeding.101.jpg
bleeding.102.jpg
bleeding.103.jpg
bleeding.104.jpg
bleeding.105.jpg
bleeding.106.jpg
bleeding.107.jpg
bleeding.108.jpg
bleeding.109.jpg
bleeding.11.jpg
bleeding.110.jpg
bleeding.111.jpg
bleeding.112.jpg
bleeding.113.jpg
bleeding.114.jpg
bleeding.115.jpg
bleeding.116.jpg
bleeding.117.jpg
bleeding.118.jpg
bleeding.119.jpg
bleeding.12.jpg
bleeding.120.jpg
bleeding.121.jpg
bleeding.122.jpg
bleeding.123.jpg
bleeding.124.jpg
bleeding.125.jpg
bleeding.126.jpg
bleeding.127.jpg
bleeding.128.jpg
bleeding.129.jpg
bleeding.13.jpg
bleeding.130.jpg
bleeding.131.jpg
bleeding.132.jpg
bleeding.133.jpg
bleeding.134.jpg
bleeding.135.jpg
bleeding.136.jpg
bleeding.137.jpg
bleeding.138.jpg
bleeding.139.jpg
bleeding.14.jpg
bleeding.140.jpg
bleeding.141.jpg
bleeding.142.jpg
bleeding.143.jpg
bleeding.144.jpg
bleeding.145.jpg
bleeding.146.jpg
bleeding.147.jpg
bleeding.148.jpg
bleeding.149.jpg
bleeding.15.jpg
bleeding.150.jpg
bleeding.151.jpg
bleed

In [14]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "nonbleeding.txt"
current_class = 'nonbleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "Brightness"

# Strengthen or weaken the contrast in each image.   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5))
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDnonBleeding) + '.jpg',image_aug)
    startIDnonBleeding += 1
print('Done')

nonbleeding.1.jpg
nonbleeding.10.jpg
nonbleeding.100.jpg
nonbleeding.101.jpg
nonbleeding.102.jpg
nonbleeding.103.jpg
nonbleeding.104.jpg
nonbleeding.105.jpg
nonbleeding.106.jpg
nonbleeding.107.jpg
nonbleeding.108.jpg
nonbleeding.109.jpg
nonbleeding.11.jpg
nonbleeding.110.jpg
nonbleeding.111.jpg
nonbleeding.112.jpg
nonbleeding.113.jpg
nonbleeding.114.jpg
nonbleeding.115.jpg
nonbleeding.116.jpg
nonbleeding.117.jpg
nonbleeding.118.jpg
nonbleeding.119.jpg
nonbleeding.12.jpg
nonbleeding.120.jpg
nonbleeding.121.jpg
nonbleeding.122.jpg
nonbleeding.123.jpg
nonbleeding.124.jpg
nonbleeding.125.jpg
nonbleeding.126.jpg
nonbleeding.127.jpg
nonbleeding.128.jpg
nonbleeding.129.jpg
nonbleeding.13.jpg
nonbleeding.130.jpg
nonbleeding.131.jpg
nonbleeding.132.jpg
nonbleeding.133.jpg
nonbleeding.134.jpg
nonbleeding.135.jpg
nonbleeding.136.jpg
nonbleeding.137.jpg
nonbleeding.138.jpg
nonbleeding.139.jpg
nonbleeding.14.jpg
nonbleeding.140.jpg
nonbleeding.141.jpg
nonbleeding.142.jpg
nonbleeding.143.jpg
nonblee

In [15]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "bleeding.txt"
current_class = 'bleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "ContrastNormalization"

# Improve or worsen the contrast of images.

from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.ContrastNormalization((0.8, 1.5), per_channel=True)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDBleeding) + '.jpg',image_aug)
    startIDBleeding += 1
print('Done')

bleeding.1.jpg
bleeding.10.jpg
bleeding.100.jpg
bleeding.101.jpg
bleeding.102.jpg
bleeding.103.jpg
bleeding.104.jpg
bleeding.105.jpg
bleeding.106.jpg
bleeding.107.jpg
bleeding.108.jpg
bleeding.109.jpg
bleeding.11.jpg
bleeding.110.jpg
bleeding.111.jpg
bleeding.112.jpg
bleeding.113.jpg
bleeding.114.jpg
bleeding.115.jpg
bleeding.116.jpg
bleeding.117.jpg
bleeding.118.jpg
bleeding.119.jpg
bleeding.12.jpg
bleeding.120.jpg
bleeding.121.jpg
bleeding.122.jpg
bleeding.123.jpg
bleeding.124.jpg
bleeding.125.jpg
bleeding.126.jpg
bleeding.127.jpg
bleeding.128.jpg
bleeding.129.jpg
bleeding.13.jpg
bleeding.130.jpg
bleeding.131.jpg
bleeding.132.jpg
bleeding.133.jpg
bleeding.134.jpg
bleeding.135.jpg
bleeding.136.jpg
bleeding.137.jpg
bleeding.138.jpg
bleeding.139.jpg
bleeding.14.jpg
bleeding.140.jpg
bleeding.141.jpg
bleeding.142.jpg
bleeding.143.jpg
bleeding.144.jpg
bleeding.145.jpg
bleeding.146.jpg
bleeding.147.jpg
bleeding.148.jpg
bleeding.149.jpg
bleeding.15.jpg
bleeding.150.jpg
bleeding.151.jpg
bleed

In [16]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "nonbleeding.txt"
current_class = 'nonbleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "ContrastNormalization"

# Improve or worsen the contrast of images.

from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.ContrastNormalization((0.8, 1.5), per_channel=True)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDnonBleeding) + '.jpg',image_aug)
    startIDnonBleeding += 1
print('Done')

nonbleeding.1.jpg
nonbleeding.10.jpg
nonbleeding.100.jpg
nonbleeding.101.jpg
nonbleeding.102.jpg
nonbleeding.103.jpg
nonbleeding.104.jpg
nonbleeding.105.jpg
nonbleeding.106.jpg
nonbleeding.107.jpg
nonbleeding.108.jpg
nonbleeding.109.jpg
nonbleeding.11.jpg
nonbleeding.110.jpg
nonbleeding.111.jpg
nonbleeding.112.jpg
nonbleeding.113.jpg
nonbleeding.114.jpg
nonbleeding.115.jpg
nonbleeding.116.jpg
nonbleeding.117.jpg
nonbleeding.118.jpg
nonbleeding.119.jpg
nonbleeding.12.jpg
nonbleeding.120.jpg
nonbleeding.121.jpg
nonbleeding.122.jpg
nonbleeding.123.jpg
nonbleeding.124.jpg
nonbleeding.125.jpg
nonbleeding.126.jpg
nonbleeding.127.jpg
nonbleeding.128.jpg
nonbleeding.129.jpg
nonbleeding.13.jpg
nonbleeding.130.jpg
nonbleeding.131.jpg
nonbleeding.132.jpg
nonbleeding.133.jpg
nonbleeding.134.jpg
nonbleeding.135.jpg
nonbleeding.136.jpg
nonbleeding.137.jpg
nonbleeding.138.jpg
nonbleeding.139.jpg
nonbleeding.14.jpg
nonbleeding.140.jpg
nonbleeding.141.jpg
nonbleeding.142.jpg
nonbleeding.143.jpg
nonblee

In [17]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "bleeding.txt"
current_class = 'bleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "Fliplr"

# Flip/mirror input images horizontally.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Fliplr(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDBleeding) + '.jpg',image_aug)
    startIDBleeding += 1
print('Done')

bleeding.1.jpg
bleeding.10.jpg
bleeding.100.jpg
bleeding.101.jpg
bleeding.102.jpg
bleeding.103.jpg
bleeding.104.jpg
bleeding.105.jpg
bleeding.106.jpg
bleeding.107.jpg
bleeding.108.jpg
bleeding.109.jpg
bleeding.11.jpg
bleeding.110.jpg
bleeding.111.jpg
bleeding.112.jpg
bleeding.113.jpg
bleeding.114.jpg
bleeding.115.jpg
bleeding.116.jpg
bleeding.117.jpg
bleeding.118.jpg
bleeding.119.jpg
bleeding.12.jpg
bleeding.120.jpg
bleeding.121.jpg
bleeding.122.jpg
bleeding.123.jpg
bleeding.124.jpg
bleeding.125.jpg
bleeding.126.jpg
bleeding.127.jpg
bleeding.128.jpg
bleeding.129.jpg
bleeding.13.jpg
bleeding.130.jpg
bleeding.131.jpg
bleeding.132.jpg
bleeding.133.jpg
bleeding.134.jpg
bleeding.135.jpg
bleeding.136.jpg
bleeding.137.jpg
bleeding.138.jpg
bleeding.139.jpg
bleeding.14.jpg
bleeding.140.jpg
bleeding.141.jpg
bleeding.142.jpg
bleeding.143.jpg
bleeding.144.jpg
bleeding.145.jpg
bleeding.146.jpg
bleeding.147.jpg
bleeding.148.jpg
bleeding.149.jpg
bleeding.15.jpg
bleeding.150.jpg
bleeding.151.jpg
bleed

In [18]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "nonbleeding.txt"
current_class = 'nonbleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "Fliplr"

# Flip/mirror input images horizontally.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Fliplr(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDnonBleeding) + '.jpg',image_aug)
    startIDnonBleeding += 1
print('Done')

nonbleeding.1.jpg
nonbleeding.10.jpg
nonbleeding.100.jpg
nonbleeding.101.jpg
nonbleeding.102.jpg
nonbleeding.103.jpg
nonbleeding.104.jpg
nonbleeding.105.jpg
nonbleeding.106.jpg
nonbleeding.107.jpg
nonbleeding.108.jpg
nonbleeding.109.jpg
nonbleeding.11.jpg
nonbleeding.110.jpg
nonbleeding.111.jpg
nonbleeding.112.jpg
nonbleeding.113.jpg
nonbleeding.114.jpg
nonbleeding.115.jpg
nonbleeding.116.jpg
nonbleeding.117.jpg
nonbleeding.118.jpg
nonbleeding.119.jpg
nonbleeding.12.jpg
nonbleeding.120.jpg
nonbleeding.121.jpg
nonbleeding.122.jpg
nonbleeding.123.jpg
nonbleeding.124.jpg
nonbleeding.125.jpg
nonbleeding.126.jpg
nonbleeding.127.jpg
nonbleeding.128.jpg
nonbleeding.129.jpg
nonbleeding.13.jpg
nonbleeding.130.jpg
nonbleeding.131.jpg
nonbleeding.132.jpg
nonbleeding.133.jpg
nonbleeding.134.jpg
nonbleeding.135.jpg
nonbleeding.136.jpg
nonbleeding.137.jpg
nonbleeding.138.jpg
nonbleeding.139.jpg
nonbleeding.14.jpg
nonbleeding.140.jpg
nonbleeding.141.jpg
nonbleeding.142.jpg
nonbleeding.143.jpg
nonblee

In [19]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "bleeding.txt"
current_class = 'bleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "Flipud"

# Flip/mirror input images vertically.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Flipud(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDBleeding) + '.jpg',image_aug)
    startIDBleeding += 1
print('Done')

bleeding.1.jpg
bleeding.10.jpg
bleeding.100.jpg
bleeding.101.jpg
bleeding.102.jpg
bleeding.103.jpg
bleeding.104.jpg
bleeding.105.jpg
bleeding.106.jpg
bleeding.107.jpg
bleeding.108.jpg
bleeding.109.jpg
bleeding.11.jpg
bleeding.110.jpg
bleeding.111.jpg
bleeding.112.jpg
bleeding.113.jpg
bleeding.114.jpg
bleeding.115.jpg
bleeding.116.jpg
bleeding.117.jpg
bleeding.118.jpg
bleeding.119.jpg
bleeding.12.jpg
bleeding.120.jpg
bleeding.121.jpg
bleeding.122.jpg
bleeding.123.jpg
bleeding.124.jpg
bleeding.125.jpg
bleeding.126.jpg
bleeding.127.jpg
bleeding.128.jpg
bleeding.129.jpg
bleeding.13.jpg
bleeding.130.jpg
bleeding.131.jpg
bleeding.132.jpg
bleeding.133.jpg
bleeding.134.jpg
bleeding.135.jpg
bleeding.136.jpg
bleeding.137.jpg
bleeding.138.jpg
bleeding.139.jpg
bleeding.14.jpg
bleeding.140.jpg
bleeding.141.jpg
bleeding.142.jpg
bleeding.143.jpg
bleeding.144.jpg
bleeding.145.jpg
bleeding.146.jpg
bleeding.147.jpg
bleeding.148.jpg
bleeding.149.jpg
bleeding.15.jpg
bleeding.150.jpg
bleeding.151.jpg
bleed

In [20]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "nonbleeding.txt"
current_class = 'nonbleeding'

imageList = getImageList(trainImageTxtFile)
current_operation = "Flipud"

# Flip/mirror input images vertically.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Flipud(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDnonBleeding) + '.jpg',image_aug)
    startIDnonBleeding += 1
print('Done')

nonbleeding.1.jpg
nonbleeding.10.jpg
nonbleeding.100.jpg
nonbleeding.101.jpg
nonbleeding.102.jpg
nonbleeding.103.jpg
nonbleeding.104.jpg
nonbleeding.105.jpg
nonbleeding.106.jpg
nonbleeding.107.jpg
nonbleeding.108.jpg
nonbleeding.109.jpg
nonbleeding.11.jpg
nonbleeding.110.jpg
nonbleeding.111.jpg
nonbleeding.112.jpg
nonbleeding.113.jpg
nonbleeding.114.jpg
nonbleeding.115.jpg
nonbleeding.116.jpg
nonbleeding.117.jpg
nonbleeding.118.jpg
nonbleeding.119.jpg
nonbleeding.12.jpg
nonbleeding.120.jpg
nonbleeding.121.jpg
nonbleeding.122.jpg
nonbleeding.123.jpg
nonbleeding.124.jpg
nonbleeding.125.jpg
nonbleeding.126.jpg
nonbleeding.127.jpg
nonbleeding.128.jpg
nonbleeding.129.jpg
nonbleeding.13.jpg
nonbleeding.130.jpg
nonbleeding.131.jpg
nonbleeding.132.jpg
nonbleeding.133.jpg
nonbleeding.134.jpg
nonbleeding.135.jpg
nonbleeding.136.jpg
nonbleeding.137.jpg
nonbleeding.138.jpg
nonbleeding.139.jpg
nonbleeding.14.jpg
nonbleeding.140.jpg
nonbleeding.141.jpg
nonbleeding.142.jpg
nonbleeding.143.jpg
nonblee

In [21]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "bleeding.txt"
current_class = 'bleeding'
imageList = getImageList(trainImageTxtFile)

current_operation = "Rot90or270Degree"

# Rotates all images by 90 or 270 degrees. 
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Rot90([1, 3])
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDBleeding) + '.jpg',image_aug)
    startIDBleeding += 1
print('Done')

bleeding.1.jpg
bleeding.10.jpg
bleeding.100.jpg
bleeding.101.jpg
bleeding.102.jpg
bleeding.103.jpg
bleeding.104.jpg
bleeding.105.jpg
bleeding.106.jpg
bleeding.107.jpg
bleeding.108.jpg
bleeding.109.jpg
bleeding.11.jpg
bleeding.110.jpg
bleeding.111.jpg
bleeding.112.jpg
bleeding.113.jpg
bleeding.114.jpg
bleeding.115.jpg
bleeding.116.jpg
bleeding.117.jpg
bleeding.118.jpg
bleeding.119.jpg
bleeding.12.jpg
bleeding.120.jpg
bleeding.121.jpg
bleeding.122.jpg
bleeding.123.jpg
bleeding.124.jpg
bleeding.125.jpg
bleeding.126.jpg
bleeding.127.jpg
bleeding.128.jpg
bleeding.129.jpg
bleeding.13.jpg
bleeding.130.jpg
bleeding.131.jpg
bleeding.132.jpg
bleeding.133.jpg
bleeding.134.jpg
bleeding.135.jpg
bleeding.136.jpg
bleeding.137.jpg
bleeding.138.jpg
bleeding.139.jpg
bleeding.14.jpg
bleeding.140.jpg
bleeding.141.jpg
bleeding.142.jpg
bleeding.143.jpg
bleeding.144.jpg
bleeding.145.jpg
bleeding.146.jpg
bleeding.147.jpg
bleeding.148.jpg
bleeding.149.jpg
bleeding.15.jpg
bleeding.150.jpg
bleeding.151.jpg
bleed

In [22]:
##################################################
# 1. Define data augmentation operations
##################################################

trainImageTxtFile = dataPath + "nonbleeding.txt"
current_class = 'nonbleeding'
imageList = getImageList(trainImageTxtFile)

current_operation = "Rot90or270Degree"

# Rotates all images by 90 or 270 degrees. 
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Rot90([1, 3])
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##################################################
# 2. loop through images
##################################################

for img in imageList:
    print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ current_class +'/'+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class  + str(startIDnonBleeding) + '.jpg',image_aug)
    startIDnonBleeding += 1
print('Done')

nonbleeding.1.jpg
nonbleeding.10.jpg
nonbleeding.100.jpg
nonbleeding.101.jpg
nonbleeding.102.jpg
nonbleeding.103.jpg
nonbleeding.104.jpg
nonbleeding.105.jpg
nonbleeding.106.jpg
nonbleeding.107.jpg
nonbleeding.108.jpg
nonbleeding.109.jpg
nonbleeding.11.jpg
nonbleeding.110.jpg
nonbleeding.111.jpg
nonbleeding.112.jpg
nonbleeding.113.jpg
nonbleeding.114.jpg
nonbleeding.115.jpg
nonbleeding.116.jpg
nonbleeding.117.jpg
nonbleeding.118.jpg
nonbleeding.119.jpg
nonbleeding.12.jpg
nonbleeding.120.jpg
nonbleeding.121.jpg
nonbleeding.122.jpg
nonbleeding.123.jpg
nonbleeding.124.jpg
nonbleeding.125.jpg
nonbleeding.126.jpg
nonbleeding.127.jpg
nonbleeding.128.jpg
nonbleeding.129.jpg
nonbleeding.13.jpg
nonbleeding.130.jpg
nonbleeding.131.jpg
nonbleeding.132.jpg
nonbleeding.133.jpg
nonbleeding.134.jpg
nonbleeding.135.jpg
nonbleeding.136.jpg
nonbleeding.137.jpg
nonbleeding.138.jpg
nonbleeding.139.jpg
nonbleeding.14.jpg
nonbleeding.140.jpg
nonbleeding.141.jpg
nonbleeding.142.jpg
nonbleeding.143.jpg
nonblee